# Introduction
The goal of this file is to test the implementation of the Speech-to-Text (STT) AI.


# 1 - Load models, modules and variables

In [1]:
#!/usr/bin/env python3
from vosk import Model, KaldiRecognizer, SetLogLevel
import sys
import os
import wave
import subprocess
import mutagen
import json
from mutagen.mp3 import MP3
from mutagen.wave import WAVE

In [2]:
# # ### Install modules for jupyter notebook
# !{sys.executable} -m pip --version
# print(sys.executable)
# {sys.executable} -m pip install srt

In [3]:
sys.path.insert(0, os.path.abspath('..'))
home_dir = os.getcwd()
model = Model(home_dir + "/Vosk_Models/model")
#Download ffmpeg here: https://ffmpeg.org/download.html and set the path
FFMPEG_PATH = home_dir + "/win32-x64/ffmpeg.exe"
sample_rate = 48000
rec = KaldiRecognizer(model, sample_rate)

# 2 - Get the audios and the audio's informations

In [4]:
# get audios
audio_paths = []
audio_dir = [f.lower() for f in os.listdir(home_dir+"/audios")]
audio_dir = sorted(audio_dir)
for file in audio_dir:
    if file[0] != '.' and ".mp3" or ".wav" in file:
        audio_paths.append(file)
print("audios: ", audio_paths)

audios:  ['audio_interview.wav', 'python_example_test.wav']


In [5]:
# function to convert the information into 
# some readable format
def audio_duration(audio):
    a = WAVE(audio)
    audio_info = a.info
    og_length = int(audio_info.length)
    hours = og_length // 3600  # calculate in hours
    length = og_length % 3600
    mins = og_length // 60  # calculate in minutes
    length = og_length % 60
    seconds = length  # calculate in seconds
    print('Total Duration: {}:{}:{}'.format(hours, mins, seconds))  
    return og_length  # returns the duration in total seconds

def frame_rate_channel(audio_file_name):
    with wave.open(audio_file_name, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        channels = wave_file.getnchannels()
        print('Frame rate: ', frame_rate, ', channels count: ', channels)
        return frame_rate,channels

audio_duration(home_dir + "/audios/" + "audio_interview.wav")
frame_rate_channel(home_dir + "/audios/" + "audio_interview.wav")

Total Duration: 0:3:56
Frame rate:  48000 , channels count:  1


(48000, 1)

# 3 - Transcribe the audio

In [6]:
def transcription_vosk(audio_uri):
    file_name = audio_uri.split('/')
    wav_name = file_name[-1].split('.')

    process = subprocess.Popen(['ffmpeg', '-loglevel', 'quiet', '-i',
                                    audio_uri,
                                    '-ar', str(sample_rate), '-ac', '1', '-f', 's16le', '-'],
                                   stdout=subprocess.PIPE)

    listSentences = []
    while True:
        data = process.stdout.read(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            res = json.loads(rec.Result())
            listSentences.append(res['text'])
#             newRes = {}
#             if res['text'] != "":
#                 newRes['segments'] = []
#                 prob = 0
#                 i = 1
#                 for segment in res['result']:
#                     prob += segment["conf"]
#                     i += 1
#                     newRes['segments'].append({
#                         "start_time": segment["start"],
#                         "probability": segment["conf"],
#                             "length": segment["end"] - segment["start"],
#                             "text": segment["word"]})

#                     newRes['probability'] = prob / i
#                     listSentences.append([newRes])

    return listSentences

In [7]:
transcription_vosk(home_dir + "/audios/" + "audio_interview.wav")

["on associe le son avec l'image",
 'non',
 'beaucoup de petits soucis',
 'la balle elle gêne pas derrière la mort parce que on entend comme julien',
 'comment',
 "ah oui ah oui c'est silencieux",
 'du coup on a essayé',
 'hum bon entend pas vous êtes au quand même là bas',
 'pour plus de bêtises',
 'bah ouais',
 'déjà euh',
 'bon le',
 "le groupe va bien on reste soudé parce que malgré tout c'est pas une défaite qui nous a fait baisser la tête c'est euh c'était un match compliqué on savait que ça allait être compliqué mais euh le groupe est soudé on est unis e on prend match après match et on se relève",
 'toute la tête pour eux pour samedi quoi',
 'la copie tout',
 '',
 "oui c'est en fait de meilleur match maintenant par rapport à avant parce que ben on est plus dynamique on pense qu on a repris un peu du poil de la bête en bas on est déterminés et euh donc euh nos prestations sont plus que positive malgré malheureusement la défaite on a une éval qui",
 'supérieur à strasbourg samedi